# Adriann Liceralde
# Combine CSV - Binary NEW

## Select Directory
1. Import all necessary libraries.
2. Set the directory to the folder that contains the target dataset.
3. Select all the csv files within the folder

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

In [2]:
os.chdir('D:\\')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
#user = 'Adriann Liceralde'
user      = 'adria'
#subfolder = 'V 3.5.1 Roof-Primary'
#subfolder = 'V 3.5.2 Roof-Secondary'
subfolder = 'V Test'
os.chdir('C:\\Users\\'+str(user)+'\\Desktop\\Repo\\WirelessSensorNetwork\\Data\\'+ str(subfolder))
#os.chdir('C:\\Users\\'+str(user)+'\\Desktop\\'+str(subfolder))
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

## Process Binary Data
1. Establish the data type class.
2. Send information from each file over to a main numpy array
3. Turn numpy array into pandas datafrane

In [3]:
first  = True
mainNP = np.array([])
dt     = np.dtype( [('unixtime','>u4'), ('ws','u1'), ('wd','>u2'), ('co2','>u2'), ('temp','u1')] )

for each in all_filenames:
    filesize   = int(os.path.getsize(each))
    headersize = 0
    lines      = (filesize-headersize)//10
    print('File :' , each, '\n', 'Bytes:' , filesize, '\n', 'Lines:' , lines)
    #print(lines)

    fileNP = np.fromfile(each, dtype = dt)
    
    if (first == True):
        mainNP = fileNP
        first = False
    elif (first == False):
        mainNP = np.concatenate( (mainNP, fileNP))
        
mainDF = pd.DataFrame(mainNP)
mainDF = mainDF.assign(date = pd.to_datetime(mainDF.unixtime, unit = 's').dt.strftime('%m/%d/%Y %H:%M'))
mainDF.ws = (mainDF.ws*2.25/3/2.237).round(2)
mainDF = mainDF[['unixtime', 'date', 'ws', 'wd', 'co2', 'temp']]

File : 2021-08-17--21-29.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-17--21-59.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-17--22-29.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-17--22-59.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-17--23-29.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-17--23-59.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--00-29.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--01-00.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--01-30.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--02-00.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--02-30.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--03-00.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--03-30.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--04-00.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--04-30.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--05-00.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--05-30.csv 
 Bytes: 6000 
 Lines: 600
File : 2021-08-18--06-01.csv 
 Bytes: 6000 
 Lin

In [4]:
print(mainDF)

         unixtime              date   ws   wd  co2  temp
0      1629235772  08/17/2021 21:29  0.0  208  829    24
1      1629235775  08/17/2021 21:29  0.0  169  831    24
2      1629235778  08/17/2021 21:29  0.0  153  847    24
3      1629235781  08/17/2021 21:29  0.0  120  823    24
4      1629235784  08/17/2021 21:29  0.0  124  819    24
...           ...               ...  ...  ...  ...   ...
41445  1629360521  08/19/2021 08:08  0.0  133  317    24
41446  1629360524  08/19/2021 08:08  0.0  142  317    24
41447  1629360527  08/19/2021 08:08  0.0  128  317    24
41448  1629360530  08/19/2021 08:08  0.0  113  317    24
41449  1629360533  08/19/2021 08:08  0.0  124  317    24

[41450 rows x 6 columns]


## Export to Single CSV
1. Select the name for the finalized csv file
2. Export the pandas dataframe to a csv file into a different folder directory
3. Check to see if all the timestamps in the dataset have been accounted for.

In [ ]:
node   = subfolder[12:16]
target = ('V All')
os.chdir('C:\\Users\\'+str(user)+'\\Desktop\\Repo\\WirelessSensorNetwork\\Data\\'+ str(target))
name = 'C-'+ str(all_filenames[0])[0:-8] + str(node)+ '.csv'
mainDF.to_csv(name, index=False, encoding='utf-8-sig')

In [ ]:
os.chdir('C:\\Users\\'+str(user)+'\\Desktop\\Repo\\WirelessSensorNetwork\\Data\\V Test')
name = 'Test.csv'
#name = 'C-'+ str(all_filenames[0])[0:-8] + str(node)+ '.csv'
mainDF.to_csv(name, index=False, encoding='utf-8-sig')

In [5]:
name = 'Test.csv'
mainDF.to_csv(name, index=False, encoding='utf-8-sig')

In [ ]:
#Check if all timestamps are accounted for
first = mainNP[0][0]
last  = mainNP[-1][0]
span = np.arange(first, last+3, 3)
unix = mainDF.unixtime.to_numpy()
summation = sum(unix-span)
print(summation)